In [ ]:
#loading dataset

In [53]:
# pip install gensim

In [2]:
import os
import random
import numpy as np
import pandas as pd
import string
import re
from bs4 import BeautifulSoup
import unicodedata
from contractions import contractions_dict
from gensim.parsing.preprocessing import remove_stopwords

In [52]:
# import gensim
# gensim.parsing.preprocessing.STOPWORDS

In [131]:
temp = 'first-rate first-rate	insightful	clever charming	comical	charismatic enjoyable	uproarious	original tender	hilarious	absorbing sensitive	riveting	intriguing powerful	fascinating	pleasant surprising	dazzling	thought-provoking imaginative	legendary	unpretentious Absorbing	Big-Budget	Bland	BloodyBoring	Brutal	Cliche Ridden	Comical Dazzling	Disappointing	Distasteful	Dramatic Enjoyable	Fantasy	Fascinating	Flawed Hilarious	Insightful	Intense	Interesting Legendary	Light-Hearted	Low-Budget	Memorable Mind-Blowing	Narrative	Predictable	Quirky Realistic	Remarkable	Romantic	Satirical first-rate insightful clever charming comical charismatic enjoyable uproarious original tender hilarious absorbing sensitive riveting intriguing powerful fascinating pleasant surprising dazzling thought provoking imaginative legendary unpretentious'
temp = temp.lower().translate(str.maketrans('', '', string.punctuation))
pos_rw = temp.split()

pos_rw = set(pos_rw)

In [132]:
temp = 'second-rate violent	moronic third-rate	flawed	juvenile boring	distasteful	ordinary disgusting	senseless	static brutal	confused	disappointing bloody	silly	tired predictable	stupid	uninteresting weak	incredibly tiresome	trite uneven	cliché ridden	outdated dreadful	bland second-rate violent moronic third-rate flawed juvenile boring distasteful ordinary disgusting senseless static brutal confused disappointing bloody silly tired predictable stupid uninteresting weak incredibly tiresome trite uneven cliché ridden outdated dreadful bland'
temp = temp.lower().translate(str.maketrans('', '', string.punctuation))
neg_rw = temp.split()

neg_rw = set(neg_rw)

In [140]:
def expand_contractions(text):
    contractions_pattern = re.compile('(%s)'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text


def normalize_text(raw_text):
    # 1. Case folding
    text = raw_text.lower()   

    # 2. Remove HTML tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # 3. Remove accented characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') 

    # 4. Expanding contractions
    text = expand_contractions(text)                           
    
    # 5. Remove stopwords
    text = remove_stopwords(text)     
    
    # 6. Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation))

    return text


neg_sw = ['against','but','cannot','cant','couldnt','didnt','doesnt','dont','down','empty','hasnt','however','neither','never',
      'nevertheless','no','nobody','none','noone','nor','not','nothing','nowhere','while','without']

def count_neg_stopwords(text):
    text = text.lower()  
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    sw_count = 0
    sw_count = len([sw for sw in text.split() if sw in neg_sw])
    
    return sw_count


def count_pos_rw(text):
    text = text.lower()  
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    pos_rw_count = 0
    pos_rw_count = len([sw for sw in text.split() if sw in pos_rw])
    
    return pos_rw_count


def count_neg_rw(text):
    text = text.lower()  
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    neg_rw_count = 0
    neg_rw_count = len([sw for sw in text.split() if sw in neg_rw])
    
    return neg_rw_count

In [139]:
def preprocess(data_dir):
    
    train = pd.read_csv(os.path.join(data_dir,"train.csv"))
    test = pd.read_csv(os.path.join(data_dir,"test.csv"))
    
    train['count_neg_stopwords'] = train['text'].apply(count_neg_stopwords)
    test['count_neg_stopwords'] = test['text'].apply(count_neg_stopwords)
    
#     train['count_pos_representative_words'], train['count_neg_representative_words'] = train['text'].apply(count_rw)
#     test['count_pos_representative_words'], test['count_neg_representative_words'] = train['text'].apply(count_rw)
    train['count_pos_representative_words'] = train['text'].apply(count_pos_rw)
    train['count_neg_representative_words'] = train['text'].apply(count_neg_rw)
    test['count_pos_representative_words'] = test['text'].apply(count_pos_rw)
    test['count_neg_representative_words'] = test['text'].apply(count_neg_rw)
    
    train['text'] = train['text'].apply(normalize_text)
    test['text'] = test['text'].apply(normalize_text)

    return train, test

In [141]:
data_dir = "C:/Users/Yinan/Desktop/CS4248/Project/preprocessed data"
train_data, test_data = preprocess(data_dir)

In [142]:
train_data

,text,sentiment,rating,count_neg_stopwords,count_pos_representative_words,count_neg_representative_words
0,sense film time else based reviews opening sce...,0,3,22,1,0
1,movie starred totally forgotten star 1930s jac...,0,3,13,2,1
2,strong movie historical epic perspective story...,1,9,3,0,0
3,eddie murphy spends time looking lost children...,1,7,1,1,1
4,thelma ritter steal picture finished watching ...,1,10,15,1,1
...,...,...,...,...,...,...
24995,bought dvd sunset saw time week ago having saw...,1,9,22,4,1
24996,meant good way believe me true life aint oiris...,1,7,3,0,0
24997,interesting concept vampirism having virusbut ...,0,4,6,1,0
24998,yeah stupidity finish watching bad taste mouth...,0,2,11,0,1


In [89]:
test_data.shape

(25000, 3)

In [143]:
outputpath='C:/Users/Yinan/Desktop/CS4248/Project/preprocessed data/train.csv'
train_data.to_csv(outputpath,sep=',',index=False,header=True)

In [8]:
# train_data=pd.read_csv('D://桌面/nlpproject/dataset/train.csv')

In [10]:
# train_data.shape

(25000, 2)

In [144]:
outputpath='C:/Users/Yinan/Desktop/CS4248/Project/preprocessed data/test.csv'
test_data.to_csv(outputpath,sep=',',index=False,header=True)

In [92]:
test_data=pd.read_csv('C:/Users/Yinan/Desktop/CS4248/Project/preprocessed data/test.csv')

In [93]:
test_data.shape

(25000, 3)

In [145]:
# use the dataset where stopwords and punctuations have been removed
data_dir = "C:/Users/Yinan/Desktop/CS4248/Project/preprocessed data"
train = pd.read_csv(os.path.join(data_dir,"train.csv"))
test = pd.read_csv(os.path.join(data_dir,"test.csv"))

In [149]:
def sentence_len(text):
    length = len(text.split())
    return length

In [150]:
train['text_len'] = train['text'].apply(sentence_len)
test['text_len'] = test['text'].apply(sentence_len)

In [152]:
train

,text,sentiment,rating,text_len
0,sense film time else based reviews opening sce...,0,3,186
1,movie starred totally forgotten star 1930s jac...,0,3,312
2,strong movie historical epic perspective story...,1,9,38
3,eddie murphy spends time looking lost children...,1,7,52
4,thelma ritter steal picture finished watching ...,1,10,115
...,...,...,...,...
24995,bought dvd sunset saw time week ago having saw...,1,9,272
24996,meant good way believe me true life aint oiris...,1,7,88
24997,interesting concept vampirism having virusbut ...,0,4,81
24998,yeah stupidity finish watching bad taste mouth...,0,2,110


In [159]:
test

,text,sentiment,rating,text_len
0,yes 1939robert donatgreer garson version bestp...,1,8,84
1,slow odd film drags plods i mean plods disappo...,0,4,42
2,mountainous woods young happy campers warning ...,1,7,165
3,roy scheider steve mcqueen clint eastwood is s...,1,9,67
4,enjoyed mr magoo cartoons saw growing up enjoy...,0,1,59
...,...,...,...,...
24995,opie tom gilsonwas brotherso went movie looked...,0,1,61
24996,nightscream tv movie bound pretty dire especia...,0,3,69
24997,lets thing straight gets 7 10 normal scale bad...,1,7,106
24998,slipknot hardcore rock band des moines iowa ba...,1,10,73


In [168]:
frame = [train, test]
total = pd.concat(frame)
total.reset_index(drop=True,inplace=True)
total

,text,sentiment,rating,text_len
0,sense film time else based reviews opening sce...,0,3,186
1,movie starred totally forgotten star 1930s jac...,0,3,312
2,strong movie historical epic perspective story...,1,9,38
3,eddie murphy spends time looking lost children...,1,7,52
4,thelma ritter steal picture finished watching ...,1,10,115
...,...,...,...,...
49995,opie tom gilsonwas brotherso went movie looked...,0,1,61
49996,nightscream tv movie bound pretty dire especia...,0,3,69
49997,lets thing straight gets 7 10 normal scale bad...,1,7,106
49998,slipknot hardcore rock band des moines iowa ba...,1,10,73


In [169]:
print(round(total["text_len"].mean()))
print(round(total["text_len"].max()))

109
1345
